In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [2]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [4]:
fea_number=5
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A6_019.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A6_016', 'Input_A6_018', 'Input_A2_019', 'Input_A5_016',
       'Input_A5_018'], dtype=object)

## Linear Regression Baseline

In [5]:
lr_rmse_ave_fea(skew_data,5)

train_rmse: 0.008211383834753831
test_rmse: 0.008484609156697588
test_r2: 0.5902412124594325


## Lasso

In [6]:
laso_rmse_ave(skew_data[title],y,9e-06)

test_rmse_ave: 0.008357584368790753
[0.009133612995377278, 0.008401965664511487, 0.008360309115069287, 0.0079248005860788, 0.007920355237441388, 0.007269583594219113, 0.008664188991591125, 0.008184205750250476, 0.008548610577551753, 0.009168211175816805]


test_r2_ave: 0.589662801731906
[0.5548348330989297, 0.5762816418576069, 0.6555186409666978, 0.6041798614364363, 0.6106034146512781, 0.6762688727993693, 0.5532505480384449, 0.6022728781080777, 0.5476756441726937, 0.515741682189526]


## ElasticNet

In [7]:
ElasticNet_rmse_ave(skew_data[title],y,[0.06],0.01)

test_rmse_ave: 0.008500144100729074
[0.009239720863357928, 0.00855570446120569, 0.008723719546023943, 0.00810712304466555, 0.007925049879982009, 0.00754658350969298, 0.00877794051312177, 0.008450671815003561, 0.008519105980184852, 0.009155821394052448]


test_r2_ave: 0.5762248988118045
[0.544431523634133, 0.5606334149045594, 0.6249195351771477, 0.585757427619706, 0.6101416627476327, 0.6511279685717783, 0.5414428587716036, 0.5759524077218412, 0.5507925524357526, 0.5170496365338904]


## SVR

In [8]:
model=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,5,model)

train_rmse: 0.00832192196962971
test_rmse: 0.00853103281802025
test_r2: 0.5846668707032916


## XGB

In [9]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.12,
              booster = 'dart', 
              n_estimators = 80, 
              max_depth = 2, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.65,
              colsample_bytree = 0.35,
              reg_alpha =  0.05,
              reg_lambda = 0.7)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.008468798002314306
[0.009219810871358914, 0.008367012772851368, 0.00855466200712067, 0.008356555596579225, 0.007635318127563188, 0.007618575553007027, 0.009083049699047703, 0.008463458302336276, 0.008562927448375876, 0.008826609644902818]


test_r2_ave: 0.57889444121388
[0.5463927500226708, 0.5797997176624095, 0.6393160927236361, 0.5598752262778078, 0.6381262415512368, 0.6444399594078898, 0.5090112113397509, 0.5746682065865962, 0.5461593060315195, 0.5511557005352824]


## RandomForest

In [10]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 3,
            min_samples_split = 3,
            max_leaf_nodes = 7,
            min_samples_leaf =3,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.008684580834261145
[0.00948147826998762, 0.00868495326524994, 0.009033860644439293, 0.00829344777636391, 0.00768248733180003, 0.007201134644908318, 0.009189599518867303, 0.00883100723126776, 0.009075970371371266, 0.009371869288356007]


test_r2_ave: 0.5566273316471476
[0.5202797286402128, 0.547258351817449, 0.5977761405801938, 0.5664976765584185, 0.6336412883400799, 0.682336547132161, 0.4974244401387149, 0.5369236405715301, 0.490146922797003, 0.4939885798957123]


In [11]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=9e-06)
elastic_mod=ElasticNet(alpha=[0.06], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.12,
              booster = 'dart', 
              n_estimators = 80, 
              max_depth = 2, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.65,
              colsample_bytree = 0.35,
              reg_alpha =  0.05,
              reg_lambda = 0.7)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 3,
            min_samples_split = 3,
            max_leaf_nodes = 7,
            min_samples_leaf =3,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.00831300638181037
[0.009055630675591292, 0.00832153341312837, 0.008435149727213296, 0.007962181825554174, 0.007631672856303885, 0.007168703375756507, 0.008700995643689527, 0.008258277566054255, 0.008532053207265333, 0.009063865527547073]


test_r2_ave: 0.594079855564021
[0.5624039768587399, 0.5843553463887915, 0.6493235133203606, 0.6004368917212275, 0.6384716922838876, 0.6851913834090281, 0.5494467809540613, 0.5950409761381714, 0.5494261155403286, 0.5267018790256137]


In [15]:

stack_mod = StackingRegressor(regressors=[vote_mod, xgb_mod], 
                           meta_regressor=lasso_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.008357582994405668
[0.009133616222834227, 0.008401961223996357, 0.00836031127681961, 0.00792480476193436, 0.007920350472816862, 0.007269553029887722, 0.008664197510854498, 0.00818420975346697, 0.008548623942348425, 0.009168201749097653]


test_r2_ave: 0.5896629059852703
[0.5548345184914099, 0.5762820897354863, 0.6555184628195085, 0.6041794442932806, 0.6106038831474268, 0.6762715949923326, 0.5532496694846375, 0.6022724890200678, 0.5476742298544917, 0.5157426780140602]


In [17]:
ave(skew_data[title],y,0.15,stack_mod,0.7,vote_mod,0.15,lasso_mod)

test_rmse_ave: 0.00831645392270462
[0.009068545117622083, 0.008335943915290845, 0.008403093795931654, 0.00794134400017723, 0.007709578278573133, 0.007187451681672925, 0.008676201936340092, 0.008226844005429015, 0.008526967278019673, 0.009088569217989565]


test_r2_ave: 0.5937384646495314
[0.5611549554693973, 0.5829145459843654, 0.6519837865226765, 0.6025255481820395, 0.6310529329485794, 0.6835425925871601, 0.5520108481610482, 0.5981179078809091, 0.5499631266997514, 0.5241184020593872]
